In [26]:
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
import numpy as np

In [27]:
(X_train, Y_train), (X_test, Y_test) = K.datasets.cifar10.load_data()

In [ ]:
Y_train.shape

In [ ]:
Y_train[0]

In [30]:
def preprocessing_data(X, Y):
  x = tf.keras.applications.inception_v3.preprocess_input(X)
  y = K.utils.to_categorical(Y, 10)
  return x, y

In [31]:
X_train, Y_train = preprocessing_data(X_train, Y_train)
X_test, Y_test = preprocessing_data(X_test, Y_test)

In [ ]:
Y_train[0]

In [33]:
base_model = K.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(299, 299, 3),
    classifier_activation='softmax'
)

In [ ]:
base_model.summary()

In [35]:
inputs = K.Input(shape=(32,32,3))
input = K.layers.Lambda(lambda image : tf.image.resize(image, (299, 299)))(inputs)

x = base_model(input, training=False)

x = K.layers.GlobalAveragePooling2D()(x)

x = K.layers.Dense(300, activation='relu')(x)
x = K.layers.Dropout(0.3)(x)
outputs = K.layers.Dense(10, activation='softmax')(x)

model = K.Model(inputs, outputs)

In [36]:
lr_schedule = K.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.5)

In [37]:
base_model.trainable = False
optimizer = K.optimizers.SGD(learning_rate=lr_schedule)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])

In [38]:
history = model.fit(X_train, Y_train, 
                    validation_data=(X_test, Y_test), 
                    batch_size=128, epochs=10, verbose=1)

Epoch 1/10
391/391 [==============================] - 452s 1s/step - loss: 2.0112 - acc: 0.3181 - val_loss: 1.6271 - val_acc: 0.6312
Epoch 2/10
391/391 [==============================] - 433s 1s/step - loss: 1.5231 - acc: 0.5693 - val_loss: 1.3051 - val_acc: 0.7176
Epoch 3/10
391/391 [==============================] - 433s 1s/step - loss: 1.2901 - acc: 0.6461 - val_loss: 1.1362 - val_acc: 0.7468
Epoch 4/10
391/391 [==============================] - 433s 1s/step - loss: 1.1666 - acc: 0.6800 - val_loss: 1.0383 - val_acc: 0.7612
Epoch 5/10
391/391 [==============================] - 433s 1s/step - loss: 1.0905 - acc: 0.6992 - val_loss: 0.9777 - val_acc: 0.7680
Epoch 6/10
391/391 [==============================] - 433s 1s/step - loss: 1.0406 - acc: 0.7120 - val_loss: 0.9379 - val_acc: 0.7737
Epoch 7/10
391/391 [==============================] - 433s 1s/step - loss: 1.0055 - acc: 0.7196 - val_loss: 0.9108 - val_acc: 0.7776
Epoch 8/10
391/391 [==============================] - 433s 1s/step - 

In [39]:
results = model.evaluate(X_test, Y_test)
print("test loss, test acc : ", results)

313/313 [==============================] - 84s 267ms/step - loss: 0.8681 - acc: 0.7816
test loss, test acc :  [0.868076741695404, 0.7815999984741211]


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(base_model, to_file='model.png')

In [ ]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

In [42]:
for layer in base_model.layers[:150]:
  layer.trainable=False

for layer in base_model.layers[150:]:
  layer.trainable=True

In [43]:
optimizer=K.optimizers.SGD(1e-3)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])

In [44]:
history = model.fit(X_train, Y_train, 
                    validation_data=(X_test, Y_test), 
                    batch_size=128, epochs=10, verbose=1)

Epoch 1/10
391/391 [==============================] - 439s 1s/step - loss: 0.8978 - acc: 0.7414 - val_loss: 0.7674 - val_acc: 0.7924
Epoch 2/10
391/391 [==============================] - 433s 1s/step - loss: 0.8131 - acc: 0.7580 - val_loss: 0.7033 - val_acc: 0.7999
Epoch 3/10
391/391 [==============================] - 432s 1s/step - loss: 0.7545 - acc: 0.7746 - val_loss: 0.6593 - val_acc: 0.8062
Epoch 4/10
391/391 [==============================] - 433s 1s/step - loss: 0.7138 - acc: 0.7796 - val_loss: 0.6272 - val_acc: 0.8096
Epoch 5/10
391/391 [==============================] - 433s 1s/step - loss: 0.6788 - acc: 0.7883 - val_loss: 0.6024 - val_acc: 0.8153
Epoch 6/10
391/391 [==============================] - 443s 1s/step - loss: 0.6564 - acc: 0.7940 - val_loss: 0.5832 - val_acc: 0.8185
Epoch 7/10
391/391 [==============================] - 436s 1s/step - loss: 0.6355 - acc: 0.7998 - val_loss: 0.5678 - val_acc: 0.8211
Epoch 8/10
391/391 [==============================] - 437s 1s/step - 

In [45]:
results = model.evaluate(X_test, Y_test)
print("test loss, test acc : ", results)

313/313 [==============================] - 84s 268ms/step - loss: 0.5339 - acc: 0.8262
test loss, test acc :  [0.5338754057884216, 0.826200008392334]
